In [1]:
import urllib.request
import re
from bs4 import BeautifulSoup
from dateutil.parser import parse
import csv
import pandas as pd
import requests

In [2]:
#---------------------------------------------------------- TO BE REMOVED ----------------------------------------------------------
#Create a set of all the countries
def extract_set_countries():

    url = 'https://www.britannica.com/topic/list-of-countries-1993160'
    page = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(page)
    
    countries = []
    for link in soup.findAll('a',href=re.compile("https://www.britannica.com/place/")):
        c = re.sub("https://www.britannica.com/place/", '',link.get('href'))
        countries.append(c)
    
    return countries

parties = ['African Group','Arab States','Environmental Integrity Group',
            'European Union','Least Developed Countries','Small Island Developing States','Umbrella Group','G-77/China']
# Clean <p> tags with tab and special characters.
def clean_tp(sentence_tp):
    s = sentence_tp
    s = re.sub("\r\n\s+"," ",s)
    s = re.sub("\r\n"," ",s)
    s = re.sub("<b>"," ",s)
    s = re.sub("</b>"," ",s)
    s = re.sub("\n"," ",s)
    s = re.sub("\s\s+"," ",s)
    return s

def extract_from_txt_sentences(txt_file):
    list_tp = open(txt_file)
    return list_tp

In [3]:
# Check if a specific word p1 in parenthesis equals to s 
def in_parenthesis(p1,s):
    p1 = '('+p1+')'
    if(p1 ==s or p1.upper() ==s  or p1.lower() == s):
        return 1 
    else:
        return 0

In [4]:
# Extract all the sentence with at least one country inside
def extract_sentences_with_one_country(list_tp_cleaned):
    sentences_countries = []
    countries = extract_set_countries()
    for line in list_tp_cleaned:
        line_slited = re.split(r';|,|\s|\"(.*)\"', line)

        for p1 in countries:
            for s in line_slited:
                if(s!=None and (p1 == s or p1.upper() == s or p1.lower() == s or in_parenthesis(p1,s))):
                    if(line not in sentences_countries):
                        sentences_countries.append(line)
                        break

    return sentences_countries 

In [15]:
# Extract all the sentence with at least one party inside
def extract_sentences_with_one_party(list_tp_cleaned,parties):
    sentences_parties = []
    for line in list_tp_cleaned:
        line_slited = re.split(r';|,|\s|\"(.*)\"', line)

        for p1 in parties:
            for s in line_slited:
                if(s!=None and (p1 == s or p1.upper() == s or p1.lower() == s or in_parenthesis(p1,s))):
                    if(line not in sentences_parties):
                        sentences_parties.append(line)
                        break

    return sentences_parties 

In [16]:
# Extract all the sentence with at least two country inside
def extract_sentences_with_two_countries(list_tp_cleaned):
    sentences_countries = []
    countries = extract_set_countries()
    for line in list_tp_cleaned:
        line_splited = re.split(r';|,|\s|\"(.*)\"', line)
        for p1 in countries:
            for s1 in line_splited:
                if(s1!=None and (p1 == s1 or p1.upper() == s1 or p1.lower() == s1 or in_parenthesis(p1,s1))):
                    for p2 in countries:
                        for s2 in line_splited :
                            if(p1!=p1 and s1!=s2 and s2!= None and (p2 == s2 or p2.upper() == s2 or p2.lower() == s2 or in_parenthesis(p2,s2))):
                                if(line not in sentences_countries):
                                    sentences_countries.append(line)
                                    break
                            break
            break
            


    return sentences_countries 

In [17]:
# Extract all the sentence with all countries and parties in uppercase
def extract_sentences_with_all_upper(list_tp_cleaned,parties):
    sentences_countries = []
    list_u = extract_set_countries() +parties
    for line in list_tp_cleaned:
        line_splited = re.split(r';|,|\s|\"(.*)\"', line)
        for p1 in list_u:
            for s in line_splited:
                if(s!=None and (p1.upper() == s)):
                        if(line not in sentences_countries):
                            print(line)
                            sentences_countries.append(line)
                            break
    return sentences_countries 

In [18]:
# Extract all the sentence with at least one word who starts with uppercase
def  extract_sentences_with_start_upper(list_tp_cleaned):
    sentences = []
    parties = ['African Group','Arab States','Environmental Integrity Group',
            'European Union','Least Developed Countries','Small Island Developing States','Umbrella Group','G-77/China']
    countries = extract_set_countries() + parties
    for line in list_tp_cleaned:
        if('/([A-Z])\w+/g' in line):
            if(line not in sentences):
                sentences.append(line)

    return sentences

In [19]:

list_tp_cleaned = extract_from_txt_sentences('list_p_tags_cleaned.txt')
parties = ['African Group','Arab States','Environmental Integrity Group',
            'European Union','Least Developed Countries','Small Island Developing States','Umbrella Group','G-77/China']

#list_one_country = extract_sentences_with_one_country(list_tp_cleaned)
#list_two_countries = extract_sentences_with_two_countries(list_tp_cleaned)
#list_all_upper = extract_sentences_with_all_upper(list_tp_cleaned,parties)
#list_start_upper = extract_sentences_with_start_upper(list_tp_cleaned)
#list_one_party = extract_sentences_with_one_party(list_tp_cleaned,parties)

In [20]:
# generate list_sentence_with_country.txt file
outF = open("list_sentence_one_party.txt", "w")
for line in list_one_party :
    # write line to output file
    outF.write(line)
    outF.write("\n")
outF.close()